In [2]:
import pandas as pd


In [7]:


# Display the result
print(df_renda[['Nome_distrito', 'average_income']].head())

df = pd.read_csv('data/population_parameters_rates_by_district.csv')
df = df[['Cod_distrito', 'Nome_distrito', 'Pop_2010', 'Pop_2020', 'Pop/ha_2010']]
# Load the second dataframe from a CSV file
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Strip leading/trailing spaces and convert to lowercase for both dataframes
df['Nome_distrito'] = df['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Check for missing values
print(df['Nome_distrito'].isnull().sum())
print(df_renda['Nome_distrito'].isnull().sum())

# Define the income intervals and their midpoints
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calculate the average income for each district
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint

df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Merge the two dataframes on the 'Nome_distrito' column
result = pd.merge(df, df_renda, on='Nome_distrito')

       Nome_distrito  average_income
0          agua rasa         12.9614
1  alto de pinheiros         21.5901
2         anhanguera          6.3032
3         aricanduva          9.8699
4        artur alvim          9.2310
0
0


In [15]:
origin = result[['Nome_distrito','Pop/ha_2010', 'average_income']]
origin['Pop/ha_2010_normalized'] = origin['Pop/ha_2010'] / origin['Pop/ha_2010'].sum()
origin['average_income_normalized'] = origin['average_income'] / origin['average_income'].sum()
origin['probabilty'] = (origin['Pop/ha_2010_normalized'] + origin['average_income_normalized']) / 2

,Nome_distrito,Pop/ha_2010,average_income,Pop/ha_2010_normalized,average_income_normalized,probabilty
0,agua rasa,123.13,12.96140,0.011757,0.011606,0.011681
1,alto de pinheiros,56.00,21.59010,0.005347,0.019332,0.012339
2,anhanguera,19.78,6.30320,0.001889,0.005644,0.003766
3,aricanduva,135.79,9.86990,0.012965,0.008837,0.010901
4,artur alvim,159.50,9.23100,0.015229,0.008265,0.011747
...,...,...,...,...,...,...
90,vila medeiros,168.73,9.36830,0.016111,0.008388,0.012249
91,vila prudente,105.29,11.57800,0.010053,0.010367,0.010210
92,vila sonia,109.54,15.08430,0.010459,0.013506,0.011983
93,sao domingos,84.84,11.19020,0.008101,0.010020,0.009060
